In [1]:
from sklearn.linear_model import LogisticRegression as lr

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn import metrics

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import math
import numpy as np
import pandas as pd
# import scipy.stats as stats
import statsmodels.api as sm

import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

import seaborn as sns
sns.set(rc={'figure.figsize':(16,10)}, font_scale=1.3)

from scipy import stats

In [4]:
df = pd.read_csv('./healthcare-dataset-stroke-data.csv')

In [5]:
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [6]:
df_na = df.dropna()

In [7]:
df_na.reset_index(inplace=True)

In [8]:
pd.isnull(df_na).any()

index                False
id                   False
gender               False
age                  False
hypertension         False
heart_disease        False
ever_married         False
work_type            False
Residence_type       False
avg_glucose_level    False
bmi                  False
smoking_status       False
stroke               False
dtype: bool

In [9]:
df_na

,index,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4904,5104,14180,Female,13.0,0,0,No,children,Rural,103.08,18.6,Unknown,0
4905,5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
4906,5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
4907,5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [10]:
df_na['ever_married'] = df_na.ever_married.map({'Yes':1, 'No':0})
# For gender, 1 means Male, 0 means Female
df_na['gender'] = df_na.gender.map({'Male':1, 'Female':0})
# For Residence_type, 1 means Urban, 0 means Rural
df_na['Residence_type'] = df_na.Residence_type.map({'Urban':1, 'Rural':0})

In [11]:
# delete the row with 'gender' value of 'Other'
df_na = df_na.dropna()
df_na.reset_index(drop=True,inplace=True)
pd.isnull(df_na).any()

index                False
id                   False
gender               False
age                  False
hypertension         False
heart_disease        False
ever_married         False
work_type            False
Residence_type       False
avg_glucose_level    False
bmi                  False
smoking_status       False
stroke               False
dtype: bool

In [12]:
df_na

,index,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,9046,1.0,67.0,0,1,1,Private,1,228.69,36.6,formerly smoked,1
1,2,31112,1.0,80.0,0,1,1,Private,0,105.92,32.5,never smoked,1
2,3,60182,0.0,49.0,0,0,1,Private,1,171.23,34.4,smokes,1
3,4,1665,0.0,79.0,1,0,1,Self-employed,0,174.12,24.0,never smoked,1
4,5,56669,1.0,81.0,0,0,1,Private,1,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4903,5104,14180,0.0,13.0,0,0,0,children,0,103.08,18.6,Unknown,0
4904,5106,44873,0.0,81.0,0,0,1,Self-employed,1,125.20,40.0,never smoked,0
4905,5107,19723,0.0,35.0,0,0,1,Self-employed,0,82.99,30.6,never smoked,0
4906,5108,37544,1.0,51.0,0,0,1,Private,0,166.29,25.6,formerly smoked,0


In [13]:
df_data = df_na.drop(columns=['index','id'])
df_data

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1.0,67.0,0,1,1,Private,1,228.69,36.6,formerly smoked,1
1,1.0,80.0,0,1,1,Private,0,105.92,32.5,never smoked,1
2,0.0,49.0,0,0,1,Private,1,171.23,34.4,smokes,1
3,0.0,79.0,1,0,1,Self-employed,0,174.12,24.0,never smoked,1
4,1.0,81.0,0,0,1,Private,1,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
4903,0.0,13.0,0,0,0,children,0,103.08,18.6,Unknown,0
4904,0.0,81.0,0,0,1,Self-employed,1,125.20,40.0,never smoked,0
4905,0.0,35.0,0,0,1,Self-employed,0,82.99,30.6,never smoked,0
4906,1.0,51.0,0,0,1,Private,0,166.29,25.6,formerly smoked,0


In [14]:
df_encoded = pd.get_dummies(df_data, columns = ['work_type','smoking_status'], \
                           prefix = {'work_type':'work_type', 'smoking_status' : 'smoking_status'}, drop_first=False)
df_encoded

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,1.0,67.0,0,1,1,1,228.69,36.6,1,0,0,1,0,0,0,1,0,0
1,1.0,80.0,0,1,1,0,105.92,32.5,1,0,0,1,0,0,0,0,1,0
2,0.0,49.0,0,0,1,1,171.23,34.4,1,0,0,1,0,0,0,0,0,1
3,0.0,79.0,1,0,1,0,174.12,24.0,1,0,0,0,1,0,0,0,1,0
4,1.0,81.0,0,0,1,1,186.21,29.0,1,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4903,0.0,13.0,0,0,0,0,103.08,18.6,0,0,0,0,0,1,1,0,0,0
4904,0.0,81.0,0,0,1,1,125.20,40.0,0,0,0,0,1,0,0,0,1,0
4905,0.0,35.0,0,0,1,0,82.99,30.6,0,0,0,0,1,0,0,0,1,0
4906,1.0,51.0,0,0,1,0,166.29,25.6,0,0,0,1,0,0,0,1,0,0


In [15]:
df_treatment = df_encoded[df_encoded['heart_disease']==1]
df_control = df_encoded[df_encoded['heart_disease']==0]

In [16]:
treat_mean = df_treatment.mean()
treat_std = df_treatment.std()
control_mean = df_control.mean()
control_std = df_control.std()

In [17]:
X_list = ['gender','age','hypertension','ever_married','Residence_type','avg_glucose_level','bmi','work_type_Govt_job',
          'work_type_Never_worked','work_type_Private','work_type_Self-employed','work_type_children','smoking_status_Unknown',
          'smoking_status_formerly smoked','smoking_status_never smoked','smoking_status_smokes']
X_descrip = ['0 to Female and 1 to Male','age of the sample','observation of hypertension',
             'Marriage states','0 to Rural and 1 to Urban','average glucose level','bmi','Whether is government job',
             'Whether is never worked','Whether job is private','Whether job is self-employed','Whether job is children',
             'Whether smoking status is unknown','Whether is formerly smoked','Whether is never smoked','Whether is smokes']

In [18]:
summary_list = []

for i in range(0, len(X_list)):
    summary_list.append([X_list[i],X_descrip[i],control_mean[X_list[i]],control_std[X_list[i]],treat_mean[X_list[i]],treat_std[X_list[i]],
                    stats.ttest_ind(df_control[X_list[i]], df_treatment[X_list[i]], equal_var=False)[0]])

summary_df = pd.DataFrame(summary_list,columns=
                          ['Label', 'Variable Description','Controls Mean','Controls STD','Treated Mean','Treated STD','t-Statistics'])
summary_df

,Label,Variable Description,Controls Mean,Controls STD,Treated Mean,Treated STD,t-Statistics
0,gender,0 to Female and 1 to Male,0.400429,0.490038,0.588477,0.493125,-5.797275
1,age,age of the sample,41.545363,22.198937,68.275720,11.706205,-32.666881
2,hypertension,observation of hypertension,0.084244,0.277784,0.238683,0.427158,-5.574922
3,ever_married,Marriage states,0.640729,0.479838,0.884774,0.319954,-11.249369
4,Residence_type,0 to Rural and 1 to Urban,0.507610,0.499996,0.502058,0.501028,0.168433
5,avg_glucose_level,average glucose level,103.730249,42.739794,135.382881,62.108697,-7.848184
6,bmi,bmi,28.820493,7.958740,30.316461,5.290212,-4.169191
7,work_type_Govt_job,Whether is government job,0.127974,0.334097,0.135802,0.343286,-0.347017
8,work_type_Never_worked,Whether is never worked,0.004716,0.068518,0.000000,0.000000,4.701011
9,work_type_Private,Whether job is private,0.572562,0.494760,0.572016,0.495808,0.016712


In [19]:
df = df_encoded
df

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,1.0,67.0,0,1,1,1,228.69,36.6,1,0,0,1,0,0,0,1,0,0
1,1.0,80.0,0,1,1,0,105.92,32.5,1,0,0,1,0,0,0,0,1,0
2,0.0,49.0,0,0,1,1,171.23,34.4,1,0,0,1,0,0,0,0,0,1
3,0.0,79.0,1,0,1,0,174.12,24.0,1,0,0,0,1,0,0,0,1,0
4,1.0,81.0,0,0,1,1,186.21,29.0,1,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4903,0.0,13.0,0,0,0,0,103.08,18.6,0,0,0,0,0,1,1,0,0,0
4904,0.0,81.0,0,0,1,1,125.20,40.0,0,0,0,0,1,0,0,0,1,0
4905,0.0,35.0,0,0,1,0,82.99,30.6,0,0,0,0,1,0,0,0,1,0
4906,1.0,51.0,0,0,1,0,166.29,25.6,0,0,0,1,0,0,0,1,0,0


In [20]:
Y = df.stroke

df_data = df.loc[:,df.columns !='stroke']
T = df_data.heart_disease
X = df_data.loc[:,df_data.columns !='heart_disease']

In [22]:
X_hd = ['age','hypertension','avg_glucose_level','bmi']

In [23]:
T.mean()

0.049511002444987774

In [53]:
# weighted based on size ratio of treatment group and control group
model_initial = Pipeline([('scaler', StandardScaler()),('logistic_classifier', lr(class_weight={0:1,1:8.5}))])
model_initial.fit(X.loc[:,X_hd], T)

predictions_binary = model_initial.predict(X.loc[:,X_hd])
print('Accuracy: {:.4f}\n'.format(metrics.accuracy_score(T, predictions_binary)))
print('Confusion matrix:\n{}\n'.format(metrics.confusion_matrix(T, predictions_binary)))
print('F1 score is: {:.4f}'.format(metrics.f1_score(T, predictions_binary)))

Accuracy: 0.8462

Confusion matrix:
[[3998  667]
 [  88  155]]

F1 score is: 0.2911


In [54]:
likelihood_ratio_table = []
X_l = X_hd.copy()

while not likelihood_ratio_table or max([x for x in likelihood_ratio_table[-1].values() if x != '-']) >= 1:
    if likelihood_ratio_table:
        max_likelihood_ratio = max([x for x in likelihood_ratio_table[-1].values() if x != '-'])
        next_label = [key for key in likelihood_ratio_table[-1] if likelihood_ratio_table[-1][key] == max_likelihood_ratio][0]
        X_l.append(next_label)
    model = Pipeline([('scaler', StandardScaler()),('logistic_classifier', lr(class_weight={0:1,1:8.5}))])
    model.fit(X.loc[:,X_l], T)
    step = {}
    for x in X.columns:
        if x in X_l:
            step[x] = '-'
        else:
            X_new = X_l.copy()
            X_new.append(x)
            model_new = Pipeline([('scaler', StandardScaler()),('logistic_classifier', lr(class_weight={0:1,1:8.5}))])
            model_new.fit(X.loc[:,X_new], T)
            
            step[x] = -2*((-metrics.log_loss(T, model.predict_proba(X.loc[:,X_l]))*len(T))
                      -(-metrics.log_loss(T, model_new.predict_proba(X.loc[:,X_new]))*len(T)))
    likelihood_ratio_table.append(step)

In [55]:
linear_terms_list = []
for x in X.columns:
    row = [step[x] for step in likelihood_ratio_table]
    row.insert(0, x)
    linear_terms_list.append(row)
    
columns = ['Label']
for i in range(0,len(likelihood_ratio_table)):
    step = 'Step ' + str(i+1)
    columns.append(step)
    
linear_terms_df = pd.DataFrame(linear_terms_list,columns = columns)
linear_terms_df

,Label,Step 1,Step 2,Step 3,Step 4,Step 5,Step 6
0,gender,84.534295,-,-,-,-,-
1,age,-,-,-,-,-,-
2,hypertension,-,-,-,-,-,-
3,ever_married,1.168323,2.996465,3.461215,-,-,-
4,Residence_type,-0.341327,-0.847312,-0.62986,-0.37917,-0.170479,-0.266482
5,avg_glucose_level,-,-,-,-,-,-
6,bmi,-,-,-,-,-,-
7,work_type_Govt_job,-1.326113,-1.333798,-1.208561,-1.162271,-1.466388,-0.746577
8,work_type_Never_worked,0.153683,0.168043,0.117838,0.132931,0.16078,0.156469
9,work_type_Private,3.569593,3.040585,2.682737,3.01767,2.913253,-


In [56]:
X_l

['age',
 'hypertension',
 'avg_glucose_level',
 'bmi',
 'gender',
 'smoking_status_smokes',
 'ever_married',
 'smoking_status_formerly smoked',
 'work_type_Private']

In [57]:
model_linear = Pipeline([('scaler', StandardScaler()),('logistic_classifier', lr(class_weight={0:1,1:8.5}))])
model_linear.fit(X.loc[:,X_l], T)

predictions_binary = model_linear.predict(X.loc[:,X_l])
print('Accuracy: {:.4f}\n'.format(metrics.accuracy_score(T, predictions_binary)))
print('Confusion matrix:\n{}\n'.format(metrics.confusion_matrix(T, predictions_binary)))
print('F1 score is: {:.4f}'.format(metrics.f1_score(T, predictions_binary)))

Accuracy: 0.8553

Confusion matrix:
[[4039  626]
 [  84  159]]

F1 score is: 0.3093


In [58]:
X_copy = X.loc[:,X_l]

for i in range(0, len(X_l)):
    j = i
    while j < len(X_l):
        X_copy[X_l[i]+'*'+X_l[j]] = X_copy[X_l[i]] * X_copy[X_l[j]]
        j += 1

X_copy

,age,hypertension,avg_glucose_level,bmi,gender,smoking_status_smokes,ever_married,smoking_status_formerly smoked,work_type_Private,age*age,...,smoking_status_smokes*smoking_status_smokes,smoking_status_smokes*ever_married,smoking_status_smokes*smoking_status_formerly smoked,smoking_status_smokes*work_type_Private,ever_married*ever_married,ever_married*smoking_status_formerly smoked,ever_married*work_type_Private,smoking_status_formerly smoked*smoking_status_formerly smoked,smoking_status_formerly smoked*work_type_Private,work_type_Private*work_type_Private
0,67.0,0,228.69,36.6,1.0,0,1,1,1,4489.0,...,0,0,0,0,1,1,1,1,1,1
1,80.0,0,105.92,32.5,1.0,0,1,0,1,6400.0,...,0,0,0,0,1,0,1,0,0,1
2,49.0,0,171.23,34.4,0.0,1,1,0,1,2401.0,...,1,1,0,1,1,0,1,0,0,1
3,79.0,1,174.12,24.0,0.0,0,1,0,0,6241.0,...,0,0,0,0,1,0,0,0,0,0
4,81.0,0,186.21,29.0,1.0,0,1,1,1,6561.0,...,0,0,0,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4903,13.0,0,103.08,18.6,0.0,0,0,0,0,169.0,...,0,0,0,0,0,0,0,0,0,0
4904,81.0,0,125.20,40.0,0.0,0,1,0,0,6561.0,...,0,0,0,0,1,0,0,0,0,0
4905,35.0,0,82.99,30.6,0.0,0,1,0,0,1225.0,...,0,0,0,0,1,0,0,0,0,0
4906,51.0,0,166.29,25.6,1.0,0,1,1,1,2601.0,...,0,0,0,0,1,1,1,1,1,1


In [59]:
likelihood_ratio_table = []
X_2 = X_l.copy()

while not likelihood_ratio_table or max([x for x in likelihood_ratio_table[-1].values() if x != '-']) >= 2.71:
    if likelihood_ratio_table:
        max_likelihood_ratio = max([x for x in likelihood_ratio_table[-1].values() if x != '-'])
        next_label = [key for key in likelihood_ratio_table[-1] if likelihood_ratio_table[-1][key] == max_likelihood_ratio][0]
        X_2.append(next_label)
    model = Pipeline([('scaler', StandardScaler()),('logistic_classifier', lr(class_weight={0:1,1:8.5}))])
    model.fit(X_copy.loc[:,X_2], T)
    step = {}
    for x in X_copy.columns:
        if x in X_2:
            step[x] = '-'
        else:
            X_new = X_2.copy()
            X_new.append(x)
            model_new = Pipeline([('scaler', StandardScaler()),('logistic_classifier', lr(class_weight={0:1,1:8.5}))])
            model_new.fit(X_copy.loc[:,X_new], T)
            
            step[x] = -2*((-metrics.log_loss(T, model.predict_proba(X_copy.loc[:,X_2]))*len(T))
                      -(-metrics.log_loss(T, model_new.predict_proba(X_copy.loc[:,X_new]))*len(T)))
    likelihood_ratio_table.append(step)

In [60]:
final_terms_list = []
for x in X_copy.columns:
    row = [step[x] for step in likelihood_ratio_table]
    row.insert(0, x)
    final_terms_list.append(row)
    
final_columns = ['Label']
for i in range(0,len(likelihood_ratio_table)):
    step = 'Step ' + str(i+1)
    final_columns.append(step)
    
final_terms_df = pd.DataFrame(final_terms_list,columns = final_columns)
final_terms_df

,Label,Step 1,Step 2,Step 3,Step 4,Step 5,Step 6,Step 7,Step 8,Step 9,Step 10,Step 11
0,age,-,-,-,-,-,-,-,-,-,-,-
1,hypertension,-,-,-,-,-,-,-,-,-,-,-
2,avg_glucose_level,-,-,-,-,-,-,-,-,-,-,-
3,bmi,-,-,-,-,-,-,-,-,-,-,-
4,gender,-,-,-,-,-,-,-,-,-,-,-
5,smoking_status_smokes,-,-,-,-,-,-,-,-,-,-,-
6,ever_married,-,-,-,-,-,-,-,-,-,-,-
7,smoking_status_formerly smoked,-,-,-,-,-,-,-,-,-,-,-
8,work_type_Private,-,-,-,-,-,-,-,-,-,-,-
9,age*age,7.734767,5.735081,4.661751,3.992416,4.35369,3.561298,-,-,-,-,-


In [64]:
X_2

['age',
 'hypertension',
 'avg_glucose_level',
 'bmi',
 'gender',
 'smoking_status_smokes',
 'ever_married',
 'smoking_status_formerly smoked',
 'work_type_Private',
 'bmi*bmi',
 'avg_glucose_level*work_type_Private',
 'avg_glucose_level*gender',
 'gender*work_type_Private',
 'age*bmi',
 'age*age',
 'age*ever_married',
 'hypertension*smoking_status_formerly smoked',
 'hypertension*ever_married',
 'gender*ever_married']

In [65]:
model_final = Pipeline([('scaler', StandardScaler()),('logistic_classifier', lr(class_weight={0:1,1:8.5}))])
model_final.fit(X_copy.loc[:,X_2], T)

predictions_binary = model_final.predict(X_copy.loc[:,X_2])
print('Accuracy: {:.4f}\n'.format(metrics.accuracy_score(T, predictions_binary)))
print('Confusion matrix:\n{}\n'.format(metrics.confusion_matrix(T, predictions_binary)))
print('F1 score is: {:.4f}'.format(metrics.f1_score(T, predictions_binary)))

Accuracy: 0.8608

Confusion matrix:
[[4059  606]
 [  77  166]]

F1 score is: 0.3271


In [66]:
treat_index = []
control_index = []
for i in range(0,len(T)):
    if T[i] == 1:
        treat_index.append(i)
    elif T[i] == 0:
        control_index.append(i)

In [67]:
predictions = model_final.predict_proba(X_copy.loc[:,X_2])

ate_ipw = sum([T[i]*Y[i]/predictions[i][1] for i in treat_index]) / sum([T[i]/predictions[i][1] for i in treat_index]) 
- sum([(1-T[i])*Y[i]/(1-predictions[i][1]) for i in control_index]) / sum([(1-T[i])/(1-predictions[i][1]) for i in control_index])

-0.0662223324708259